In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pickle

2024-12-15 19:34:02.293014: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 19:34:02.864731: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
class MTGDeckGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(MTGDeckGenerator, self).__init__()
        self.embedding = layers.Embedding(vocab_size, embedding_dim)
        self.gru = layers.GRU(hidden_dim, return_state=True, return_sequences=True)
        self.fc = layers.Dense(vocab_size)

    def call(self, commander_card):
        embedded = self.embedding(commander_card)  
        output, _ = self.gru(embedded)  
        logits = self.fc(output)  
        return logits

    def generate_deck(self, commander_card, num_cards_to_generate):
        embedded = self.embedding(commander_card)
        hidden = tf.zeros((1, self.gru.units))
        generated_deck = []
        current_input = embedded

        for _ in range(num_cards_to_generate):
            output, hidden = self.gru(current_input, initial_state=hidden)
            logits = self.fc(output[:, -1, :])
            next_card = tf.argmax(logits, axis=-1)
            generated_deck.append(next_card.numpy()[0])
            current_input = self.embedding(tf.expand_dims(next_card, axis=0))

        return generated_deck

In [3]:
# Tokenizing the dataset
def tokenize_dataset(decks):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True,
                             lower=True,
                             filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    
    tokenizer.fit_on_texts(decks)
    tokenized_decks = tokenizer.texts_to_sequences(decks)
    return tokenized_decks, tokenizer

# Training the model
def train_model(model, tokenized_decks, epochs=10, batch_size=32):
    input_cards = []
    target_cards = []
    
    for deck in tokenized_decks:
        input_cards.append(deck[:-1])
        target_cards.append(deck[1:])
    
    input_cards = tf.keras.preprocessing.sequence.pad_sequences(input_cards, padding='post')
    target_cards = tf.keras.preprocessing.sequence.pad_sequences(target_cards, padding='post')

    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    model.fit(input_cards, target_cards, epochs=epochs, batch_size=batch_size)

In [4]:
def load_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)

    return data

In [11]:
# Saving the model
def save_model(model, path="mtg_deck_generator_model.keras"):
    model.save(path)
    print(f"Model saved at {path}")

# Loading the model
def load_model(path="mtg_deck_generator_model.keras"):
    model = tf.keras.models.load_model(path, custom_objects={"MTGDeckGenerator": MTGDeckGenerator})
    print(f"Model loaded from {path}")
    return model


In [6]:
import string

card_dict_global = {}
def process_global_card_data(card_names):
    global card_dict_global
    formatted_list = []
    for name in card_names:
        formatted_name = ''.join(char.lower() for char in name if char not in string.punctuation and not char.isspace())
        if formatted_name not in card_dict_global:
            card_dict_global[formatted_name] = name
        formatted_list.append(formatted_name)
    return formatted_list

card_list = ['Giada, Font of Hope', 'Black Lotus', 'Time Walk']
formatted_list = process_global_card_data(card_list)
print("Global Dictionary:", card_dict_global)
print("Formatted List:", formatted_list)

card_list = ['Giada, Font of Hope', 'Sol Ring', 'Time Walk']
formatted_list = process_global_card_data(card_list)
print("Global Dictionary:", card_dict_global)
print("Formatted List:", formatted_list)

Global Dictionary: {'giadafontofhope': 'Giada, Font of Hope', 'blacklotus': 'Black Lotus', 'timewalk': 'Time Walk'}
Formatted List: ['giadafontofhope', 'blacklotus', 'timewalk']
Global Dictionary: {'giadafontofhope': 'Giada, Font of Hope', 'blacklotus': 'Black Lotus', 'timewalk': 'Time Walk', 'solring': 'Sol Ring'}
Formatted List: ['giadafontofhope', 'solring', 'timewalk']


In [ ]:
path = "data/mtg_decks.pkl"

card_dict_global = {}
decks = load_data(path)
better_decks = []
for deck in decks:
    better_decks.append(process_global_card_data(deck))


decks[0], better_decks[0]

(['Giada, Font of Hope',
  'Mesa Cavalier',
  'Exorcise',
  'Terramorphic Expanse',
  'Angel of the Ruins',
  'Plains',
  'Palace Sentinels',
  'Secluded Courtyard',
  'Sol Ring',
  'Fleeting Flight',
  'Divine Resilience',
  'Uncharted Haven',
  'Dusk // Dawn',
  'Hidden Grotto',
  'Sinew Dancer',
  'Plated Onslaught',
  'Evolving Wilds',
  'Banishing Light',
  'Pearl Medallion',
  'Renewed Faith',
  'Feather of Flight',
  'Vexing Bauble',
  'Crackdown',
  'Moonlit Wake',
  'Charitable Levy',
  'Dawn of a New Age',
  'Forsake the Worldly',
  "Valkyrie's Call",
  'Mace of the Valiant',
  'Dazzling Angel',
  'Command Tower',
  'Seized from Slumber',
  'Joust Through',
  'Vanguard Seraph',
  'Take Up the Shield',
  'Angel of Finality',
  'Serra Angel',
  'Decree of Justice',
  'Angelic Destiny',
  'Localized Destruction',
  'Scale Blessing',
  'Goldvein Pick',
  "Danitha, Benalia's Hope",
  'Unstable Obelisk',
  'Dazzling Theater // Prop Room',
  'Swiftfoot Boots',
  'Lyra Dawnbringer',


In [12]:
tokenized_decks, tokenizer = tokenize_dataset(better_decks[:1000])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  # Adjust based on dataset size
print(vocab_size)
embedding_dim = 128
hidden_dim = 256



model = MTGDeckGenerator(vocab_size, embedding_dim, hidden_dim)

train_model(model, tokenized_decks, epochs=1, batch_size=1028)


571
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0014 - loss: 6.3452


In [ ]:
# save_model(model)
# model = load_model()


commander_card = tf.constant([tokenizer.word_index['giadafontofhope']])
num_cards_to_generate = 65

deck = model(commander_card, num_cards_to_generate)
print(deck)

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: 65 (of type <class 'int'>)